In [ ]:
import asyncio
import sys
import os
import nest_asyncio
import json
from time import time
from typing import List, Dict
from rich.console import Console
from rich.table import Table

# Add the project root to Python path
project_root = os.path.dirname(os.path.dirname(os.getcwd()))
if project_root not in sys.path:
    sys.path.append(project_root)

from agents.sql_with_preprocess.types import AgentState
from langchain_core.messages import HumanMessage
from agents.sql_with_preprocess.main import runworkflow as arun

nest_asyncio.apply()
console = Console()

# Test queries
# queries = [
#     "Show me batting stats of Dhoni vs. Australia.",
#     "How many runs did Sachin baby score at Eden Gardens,kolkata?",
#     "Give me the top 5 players from India who've taken the most wickets.",
#     "Who has the highest strike rate in T20 matches?", 
#     "Fetch me the bowlers with the best economy rate in ODIs.",
#     "Find all matches where Kohli was dismissed for a duck.",
#     "List the grounds in England where Rohit scored centuries.",
#     "Who are the left-handed batsmen in the database?",
#     "Which teams played in the 2011 World Cup final?",
#     "Show me the biggest six-hitters in the IPL."
# ]
queries = {
    # 1. Player Search queries
    "player_search": [
        "fazal haq faroo bowling stats for srh",
        "virat Kohli's batting statistics",
        "M.S. Dhoni's career performance",
        "steve Smith's batting average by year",
        "ishant Sharma's bowling records by competition",
        "shadab khan's performance in 2023",
        "trav head  stats by year",
        "r pant career statistics",
    ],

    # 2. Batsman vs Bowler Matchup queries
    "head_to_head": [
        "v Kohli vs James Anderson head-to-head stats",
        "david Warner vs Indian bowlers statistics",
        "r Ashwin's LBW dismissals against left-handers",
        "fast bowlers' performance against rohit sharma",
        "leg spinners' bowling records against Steve Smith",
    ],

    # 3. Leaders Board queries
    "leaderboard": [
        "most caught behind dismissals batters in uppal stadium ",
        "top cover drive players stats in 1-10 overs ",
        "best bowling averages bowlers in hyderabad",
        "highest run-scorers batter in 2023",
    ],

    # 4. Venue Search queries
    "venue_stats": [
        "match statistics at Lord's Cricket Ground",
        "M Chinnaswamy Stadium batting averages in ipl",
        "mcg batting stats by year ",
        "Sydney bowling stats by batter type",
    ],

    # 5. Team Search queries
    "team_stats": [
        "Australia vs New Zealand head-to-head record",
        "csk bowling stats in cheapuak by year wise",
        "rcb batting stats",
        "india win-loss record by venue",
    ]
}
async def test_query(query: str) -> Dict:
    """Test a single query and return timing + results"""
    start_time = time()
    
    try:
        result = await arun(query)
        response = result['messages'][-1].content
        sql_query = result['sql_query']
        return {
        "query": query,
        "response": sql_query,
        "search_result":result['search_result'],
        "time_taken": round(end_time - start_time, 2)

    }
    except Exception as e:
        response = f"Error: {str(e)}"
        sql_query = response
        
    end_time = time()
    
    return {
        "query": query,
        "response": sql_query,
        "search_result":result['search_result'],
        "time_taken": round(end_time - start_time, 2)

    }

async def batch_test(queries: Dict[str, List[str]]):
    """Run all queries and display results in a table"""
    
    # Create results table
    table = Table(title="Search Agent Test Results")
    table.add_column("Category", style="magenta")
    table.add_column("Query", style="cyan")
    table.add_column("Response", style="green")
    table.add_column("Time (s)", justify="right", style="yellow")
    
    console.print("\n[bold]Starting batch testing...[/bold]\n")
    
    results = {}
    for category, category_queries in queries.items():
        console.print(f"\n[bold]{category.upper()} Queries[/bold]")
        
        category_results = []
        for query in category_queries:
            result = await test_query(query)
            result['category'] = category
            category_results.append(result)
            
            # Add to table
            table.add_row(
                category,
                result["query"],
                result["response"],
                str(result["time_taken"])
            )
        
        results[category] = category_results
    
    # Display results
    console.print(table)
    
    # Save results to file
    with open('sql_agent_test_results_2.0-flash-mistral.json', 'w') as f:
        json.dump(results, f, indent=2)
        
    console.print("\n[bold green]Results saved to sql_agent_test_results.json[/bold green]")

# Assuming 'queries' is the dictionary we created in the previous response
# Run the tests
asyncio.run(batch_test(queries))

In [1]:
#run on certain dataset
import sys
import os
project_root = os.path.dirname(os.path.dirname(os.getcwd()))
if project_root not in sys.path:
    sys.path.append(project_root)
from datetime import datetime
from csv import DictWriter
from langsmith import Client
from langsmith.evaluation import evaluate, aevaluate
from openai import AsyncOpenAI
from agents.sql_with_preprocess.main import runworkflow
import json
# client = AsyncOpenAI()
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv
load_dotenv()

async def test_agent():
    langsmith_client = Client()
    # dataset = langsmith_client.read_dataset(dataset_id="07dbb645-2988-41c8-9f85-b3a37f51bdb2")
    dataset = langsmith_client.read_dataset(dataset_id="2d5bd4c9-1c73-4064-905f-00a5c924487a")

    async def runs(inputs: dict) -> dict:
        try:
            query = inputs['inputs']["question"]
            print(f"Processing question: {query}")
            
            result = await runworkflow(query)
            
            # Extract relevant information
            response = result['messages'][-1].content
            sql_query = result.get('sql_query','')
            search_result = result.get('search_result', '')
            
            # Log to CSV
            try:
                # Load existing data if file exists
                # json_file = "logs/sql_agent_evaluation.json"  
                import os

# Ensure the logs directory exists
                log_dir = "logs"
                if not os.path.exists(log_dir):
                    os.makedirs(log_dir)

                # Your existing code for writing to the JSON file
                json_file = os.path.join(log_dir, "sql_agent_evaluation.json")
                if os.path.exists(json_file):
                    with open(json_file, 'r', encoding='utf-8') as f:
                        try:
                            existing_data = json.load(f)
                        except json.JSONDecodeError:
                            existing_data = []
                else:
                    with open(json_file, 'w', encoding='utf-8') as f:
                        json.dump([], f)
                    existing_data = []

                # Append new data
                new_entry = {
                    # "category": inputs.get("type", "category"),
                    "question": query,
                    "sequence":result['sequence'],
                    # "sql_query": sql_query,
                    # "response": response,
                    "search_result": search_result,
                    # "timestamp": datetime.now().isoformat()
                }
                existing_data.append(new_entry)

                # Write back to file
                with open(json_file, 'w', encoding='utf-8') as f:
                    json.dump(existing_data, f, indent=2, ensure_ascii=False)

            except Exception as e:
                print(f"Error writing to JSON: {e}")
                
            return {
                "output": {
                    "sequence":result['sequence'],
                    "search_result": search_result,
                    # "sql_query": sql_query,
                    # "response": response,
                    
                }
            }
        except Exception as e:
            print(f"Error in runs function: {e}")
            raise

    current_date = datetime.now().strftime("%Y%m%d")
    current_time = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

    # Convert queries dict to dataset format


    try:
        result = await aevaluate(
            runs,
            data=dataset,  # Use your structured queries instead of dataset
            experiment_prefix=f"sql_agent_{current_time}",
            max_concurrency=1,
            metadata={
                "version": f"{current_time}",
                "revision_id": "testing",
                "supervisor":"1.5-flash",
                "search":"mistral-large"

            },
        )
        print("Evaluation completed:", result)
    except Exception as e:
        print(f"Evaluation failed: {e}")

import asyncio
asyncio.run(test_agent())

in table dir C:\Users\adith\Documents\Projects\python-projects\csql-agent\agents\tables\hdata
Loading store for category 'hdata_bat_hand' from 'C:\Users\adith\Documents\Projects\python-projects\csql-agent\agents\tables\hdata\hdata_bat_hand'
Loading store for category 'hdata_bat_out' from 'C:\Users\adith\Documents\Projects\python-projects\csql-agent\agents\tables\hdata\hdata_bat_out'
Loading store for category 'hdata_bowl_kind' from 'C:\Users\adith\Documents\Projects\python-projects\csql-agent\agents\tables\hdata\hdata_bowl_kind'
Loading store for category 'hdata_bowl_style' from 'C:\Users\adith\Documents\Projects\python-projects\csql-agent\agents\tables\hdata\hdata_bowl_style'
Loading store for category 'hdata_competition' from 'C:\Users\adith\Documents\Projects\python-projects\csql-agent\agents\tables\hdata\hdata_competition'
Loading store for category 'hdata_country' from 'C:\Users\adith\Documents\Projects\python-projects\csql-agent\agents\tables\hdata\hdata_country'
Loading store fo

c:\Users\adith\Documents\Projects\python-projects\csql-agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'sql_agent_25-01-2025_12-29-30-0cae37ba' at:
https://smith.langchain.com/o/dbea2471-6360-589a-b7bc-3aa89cfaa333/datasets/2d5bd4c9-1c73-4064-905f-00a5c924487a/compare?selectedSessions=908e9671-075e-42b5-bf40-466207f520f1




0it [00:00, ?it/s]

Processing question: M Chinnaswamy Stadium batting averages in ipl
[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'attempts': 0,
 'change': '',
 'docs_schema': '',
 'execution_choice': False,
 'messages': [HumanMessage(content='M Chinnaswamy Stadium batting averages in ipl and dont execute and table name as hdata ', additional_kwargs={}, response_metadata={})],
 'query': '',
 'referenced_values_in_table': '',
 'relevant_sql_queries': '',
 'sequence': '',
 'sql_error': False,
 'sql_query': None,
 'sql_result': '',
 'table_name': None}
[0:writes] Finished step 0 with writes to 13 channels:
- messages -> [HumanMessage(content='M Chinnaswamy Stadium batting averages in ipl and dont execute and table name as hdata ', additional_kwargs={}, response_metadata={})]
- query -> ''
- execution_choice -> False
- sql_query -> None
- sql_result -> ''
- relevant_sql_queries -> ''
- sql_error -> False
- referenced_values_in_table -> '

...

Error in runs function: 429 Resource has been exhausted (e.g. check quota).
Processing question: best bowling averages bowlers in hyderabad
[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'attempts': 0,
 'change': '',
 'docs_schema': '',
 'execution_choice': False,
 'messages': [HumanMessage(content='best bowling averages bowlers in hyderabad and dont execute and table name as hdata ', additional_kwargs={}, response_metadata={})],
 'query': '',
 'referenced_values_in_table': '',
 'relevant_sql_queries': '',
 'sequence': '',
 'sql_error': False,
 'sql_query': None,
 'sql_result': '',
 'table_name': None}
[0:writes] Finished step 0 with writes to 13 channels:
- messages -> [HumanMessage(content='best bowling averages bowlers in hyderabad and dont execute and table name as hdata ', additional_kwargs={}, response_metadata={})]
- query -> ''
- execution_choice -> False
- sql_query -> None
- sql_result -> ''
- relevant_sql_q

21it [05:38, 14.97s/it]

[5:writes] Finished step 5 with writes to 2 channels:
- messages -> HumanMessage(content='Supervisor routed to __end__ agent', additional_kwargs={}, response_metadata={})
- sequence -> ' search sql __end__'
[5:checkpoint] State at the end of step 5:
{'attempts': 0,
 'change': '',
 'docs_schema': 'p_match: INTEGER(identifier) (use this to count different '
                'matches) \n'
                '\n'
                'inns: INTEGER (innings number like 1 or 2 or 3 or 4)\n'
                '\n'
                'bat: STRING (batter currently batting)\n'
                '\n'
                "p_bat: INTEGER (the batter's player id)\n"
                '\n'
                'team_bat: STRING (the team that is currently batting)\n'
                '\n'
                'bowl: STRING (the bowler currently bowling)\n'
                '\n'
                "p_bowl: INTEGER (the bowler's player id)\n"
                '\n'
                'team_bowl: STRING (the team that is currently bowling)\n'

21it [05:39, 16.17s/it]

Evaluation completed: <AsyncExperimentResults sql_agent_25-01-2025_12-29-30-0cae37ba>


In [1]:
#creating datasets

from langsmith import Client
from langsmith.evaluation import evaluate, aevaluate
from openai import AsyncOpenAI
from dotenv import load_dotenv
import json
load_dotenv()
langsmith_client = Client()
queries = {
    # 1. Player Search queries
    "player_search": [
        "fazal haq faroo bowling stats for srh",
        "virat Kohli's batting statistics",
        "M.S. Dhoni's career performance",
        "steve Smith's batting average by year",
        "ishant Sharma's bowling records by competition",
        "shadab khan's performance in 2023",
        "trav head  stats by year",
        "r pant career statistics",
    ],

    # 2. Batsman vs Bowler Matchup queries
    "head_to_head": [
        "v Kohli vs James Anderson head-to-head stats",
        "david Warner vs Indian bowlers statistics",
        "r Ashwin's LBW dismissals against left-handers",
        "fast bowlers' performance against rohit sharma",
        "leg spinners' bowling records against Steve Smith",
    ],

    # 3. Leaders Board queries
    "leaderboard": [
        "most caught behind dismissals batters in uppal stadium ",
        "top cover drive players stats in 1-10 overs ",
        "best bowling averages bowlers in hyderabad",
        "highest run-scorers batter in 2023",
    ],

    # 4. Venue Search queries
    "venue_stats": [
        "match statistics at Lord's Cricket Ground",
        "M Chinnaswamy Stadium batting averages in ipl",
        "mcg batting stats by year ",
        "Sydney bowling stats by batter type",
    ],

    # 5. Team Search queries
    "team_stats": [
        "Australia vs New Zealand head-to-head record",
        "csk bowling stats in cheapuak by year wise",
        "rcb batting stats",
        "india win-loss record by venue",
    ]
}
dataset = langsmith_client.create_dataset(
            dataset_name=f"sql_agent_evaluation_0401",
            description="cricmetric style questions"
        )

        # Add data to the dataset
evaluation_data = []
for category, category_queries in queries.items():
    for query in category_queries:
        evaluation_data.append({
            "question": query,
            "type": category
        })
for item in evaluation_data:
    langsmith_client.create_example(
        inputs=item,
        dataset_id=dataset.id
    )


In [2]:
dataset.id

UUID('07dbb645-2988-41c8-9f85-b3a37f51bdb2')